NB: We've decided to stick with v0.3.1 for now.  Will have to figure out how to keep up to date with master somewhere down the line.

Note, this requires building PyTorch from `master`, since the newest release does not contain the merge between Tensor and Variable.  See [here](https://github.com/pytorch/pytorch#from-source) for instructions.

In [1]:
import torch
from torch.autograd import Variable
from forbiddenfruit import curse

ModuleNotFoundError: No module named 'forbiddenfruit'

In [2]:
old_tensor = torch.tensor
def grid_tensor(data, dtype = None, device = None, requires_grad = False):
    print(data)
    print(dtype)
    print(device)
    print(requires_grad)
    return old_tensor(data, dtype = dtype, requires_grad = requires_grad)

old_zeros = torch.zeros
def grid_zeros(shape, dtype=None, device=None, requires_grad=False):
    print(shape)
    print(dtype)
    print(device)
    print(requires_grad)
    return old_zeros(shape,  dtype =  dtype, requires_grad = requires_grad)
    
# torch.ones(...)
# torch.empty(...)

In [3]:
Variable.add

<method 'add' of 'torch._C._VariableBase' objects>

In [4]:
# Notes -- very unstable.
# I think some of the things I'm replacing might have decorators, which could lead to some of the weird behavior
# We need to hook to these methods so that we can send out commands to workers
# Just need to be able to do so reliably
# May also need to disable the __str__ or print functionality on variables/tensors to avoid these RuntimeErrors

def hook_var():
    
    # None of the `__init__` or `new` stuff works yet
    curse(torch._C._VariableBase, "old__init__", torch._C._VariableBase.new)
    #curse(Variable, "old__init__", Variable.new)
    def _new___init__(self, *args):
        self.id = 1
        self.data_addr = 2
        print("init")
        #return self.old__init__(*args)
    #curse(torch._C._VariableBase, "__init__", _new___init__)
    curse(torch._C._VariableBase, "new", _new___init__)
    
    # `add` and `__add__` work sometimes, and even then not as expected
    curse(Variable, "old__add__", Variable.__add__)
    def _new___add__(self, other):
        print("__add__")
        #return self.old__add__(other)
    curse(Variable, "__add__", _new___add__)
   
    curse(torch._C._VariableBase, 'old_add', torch._C._VariableBase.add)
    def _new_add(self, other):
        print("add")
        #return self.old_add(other)
    curse(Variable, 'add', _new_add)
    
    # This works, usually
    torch.old_addmm = torch.addmm
    def _new_addmm(mat, mat1, mat2, beta=1, alpha=1):
        print("addmm")
        #return torch.old_addmm(beta, mat, alpha, mat1, mat2)
    torch.addmm = _new_addmm

In [5]:
def hook_torch():
    torch.tensor = grid_tensor
    torch.zeros = grid_zeros
    hook_var()

    
    #hook_long()

    # anything else we need to do to modify/override torch for the Client globally
    
    pass

In [6]:
hook_torch()

In [7]:
x = torch.tensor([2,2], requires_grad = True)

[2, 2]
None
None
True


In [8]:
x.id

AttributeError: 'Variable' object has no attribute 'id'

In [9]:
x.cuda()

RuntimeError: Overflow when unpacking long

In [10]:
x.add(x)

RuntimeError: Overflow when unpacking long

In [11]:
x + x

RuntimeError: Overflow when unpacking long

In [12]:
torch.addmm(x, x, x)

addmm


Below doesn't work -- can't subclass FloatTensor?

In [1]:
# torch.tensor(data, dtype=None, device=None, requires_grad=False)
# torch.zeros(shape, dtype=None, device=None, requires_grad=False)
# torch.ones(...)
# torch.empty(...)

In [1]:
import torch
from torch.autograd import Variable

In [15]:
class FakeVar(torch.autograd.variable):
    
    def __add__(self, other):
        return self.add(other)
    
    def add(self, other):
        print("add")


In [16]:
torch._C._VariableBase = FakeVar

In [20]:
torch.autograd.Variable = FakeVar

In [8]:
a = torch.tensor((2,2), dtype = torch.float, requires_grad = True)

(2, 2)
torch.float32
True


In [9]:
a.add(a)

__add__
__add__


RuntimeError: Overflow when unpacking long

In [23]:
type(a)

torch.autograd.variable.Variable

In [24]:
type(torch.tensor((2,2), requires_grad = True))

(2, 2)
None
True


torch.autograd.variable.Variable

In [25]:
a + a


 3.1621e-28  9.1370e-41
 3.1621e-28  9.1370e-41
[torch.FloatTensor of size (2,2)]